In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import string
import re

from wordcloud import WordCloud
from collections import Counter
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BartTokenizer, BartForConditionalGeneration
from nltk.sentiment import SentimentIntensityAnalyzer

#nltk.download("vader_lexicon")

In [ ]:
#df = pd.read_csv("Turkish.csv", sep="|", names=["Surah", "Ayat", "Turkish"])
df = pd.read_csv("quran_main_df.csv")

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
surah_list = {
1: "Fâtiha Sûresi",
2: "Bakara Sûresi",
3: "Âl-i İmrân Sûresi",
4: "Nisâ Sûresi",
5: "Mâide Sûresi",
6: "En'âm Sûresi",
7: "A'râf Sûresi",
8: "Enfâl Sûresi",
9: "Tevbe Sûresi",
10: "Yunus Sûresi",
11: "Hûd Sûresi",
12: "Yusuf Sûresi",
13: "Ra'd Sûresi",
14: "İbrahim Sûresi",
15: "Hicr Sûresi",
16: "Nahl Sûresi",
17: "İsrâ Sûresi",
18: "Kehf Sûresi",
19: "Meryem Sûresi",
20: "Tâ-Hâ Sûresi",
21: "Enbiyâ Sûresi",
22: "Hac Sûresi",
23: "Mü'minûn Sûresi",
24: "Nûr Sûresi",
25: "Furkan Sûresi",
26: "Şuarâ Sûresi",
27: "Neml Sûresi",
28: "Kasas Sûresi",
29: "Ankebût Sûresi",
30: "Rûm Sûresi",
31: "Lokman Sûresi",
32: "Secde Sûresi",
33: "Ahzâb Sûresi",
34: "Sebe' Sûresi",
35: "Fâtır Sûresi",
36: "Yâsin Sûresi",
37: "Sâffât Sûresi",
38: "Sâd Sûresi",
39: "Zümer Sûresi",
40: "Mü'min Sûresi",
41: "Fussilet Sûresi",
42: "Şûrâ Sûresi",
43: "Zuhruf Sûresi",
44: "Duhân Sûresi",
45: "Câsiye Sûresi",
46: "Ahkaf Sûresi",
47: "Muhammed Sûresi",
48: "Fetih Sûresi",
49: "Hucurât Sûresi",
50: "Kaf Sûresi",
51: "Zâriyât Sûresi",
52: "Tûr Sûresi",
53: "Necm Sûresi",
54: "Kamer Sûresi",
55: "Rahmân Sûresi",
56: "Vâkıa Sûresi",
57: "Hadid Sûresi",
58: "Mücâdele Sûresi",
59: "Haşr Sûresi",
60: "Mümtehine Sûresi",
61: "Saf Sûresi",
62: "Cum'a Sûresi",
63: "Münâfikûn Sûresi",
64: "Teğabün Sûresi",
65: "Talâk Sûresi",
66: "Tahrim Sûresi",
67: "Mülk Sûresi",
68: "Kalem Sûresi",
69: "Hâkka Sûresi",
70: "Meâric Sûresi",
71: "Nuh Sûresi",
72: "Cin Sûresi",
73: "Müzzemmil Sûresi",
74: "Müddessir Sûresi",
75: "Kıyamet Sûresi",
76: "İnsan Sûresi",
77: "Mürselât Sûresi",
78: "Nebe' Sûresi",
79: "Nâziât Sûresi",
80: "Abese Sûresi",
81: "Tekvir Sûresi",
82: "İnfitâr Sûresi",
83: "Mutaffifin Sûresi",
84: "İnşikak Sûresi",
85: "Bürûc Sûresi",
86: "Târık Sûresi",
87: "A'lâ Sûresi",
88: "Gâşiye Sûresi",
89: "Fecr Sûresi",
90: "Beled Sûresi",
91: "Şems Sûresi",
92: "Leyl Sûresi",
93: "Duhâ Sûresi",
94: "İnşirâh Sûresi",
95: "Tin Sûresi",
96: "Alak Sûresi",
97: "Kadir Sûresi",
98: "Beyyine Sûresi",
99: "Zilzâl Sûresi",
100: "Âdiyât Sûresi",
101: "Kâria Sûresi",
102: "Tekâsür Sûresi",
103: "Asr Sûresi",
104: "Hümeze Sûresi",
105: "Fil Sûresi",
106: "Kureyş Sûresi",
107: "Mâûn Sûresi",
108: "Kevser Sûresi", 
109: "Kâfirûn Sûresi",
110: "Nasr Sûresi",
111: "Tebbet Sûresi",
112: "İhlâs Sûresi",
113: "Felâk Sûresi",
114: "Nâs Sûresi"
}

In [ ]:
def surah_name(surahidx):
    surahname = surah_list.get(surahidx)
    return surahname

In [ ]:
df["Name"] = df["Surah"].apply(surah_name)

In [ ]:
surah_order = {
"Fâtiha Sûresi": 5,
"Bakara Sûresi": 87,
"Âl-i İmrân Sûresi": 89,
"Nisâ Sûresi": 92,
"Mâide Sûresi": 112,
"En'âm Sûresi": 55,
"A'râf Sûresi": 39,
"Enfâl Sûresi": 88,
"Tevbe Sûresi": 113,
"Yunus Sûresi": 51,
"Hûd Sûresi": 52,
"Yusuf Sûresi": 53,
"Ra'd Sûresi": 96,
"İbrahim Sûresi": 72,
"Hicr Sûresi": 54,
"Nahl Sûresi": 70,
"İsrâ Sûresi": 50,
"Kehf Sûresi": 69,
"Meryem Sûresi": 44,
"Tâ-Hâ Sûresi": 45,
"Enbiyâ Sûresi": 73,
"Hac Sûresi": 103,
"Mü'minûn Sûresi": 74,
"Nûr Sûresi": 102,
"Furkan Sûresi": 42,
"Şuarâ Sûresi": 47,
"Neml Sûresi": 48,
"Kasas Sûresi": 49,
"Ankebût Sûresi": 85,
"Rûm Sûresi": 84,
"Lokman Sûresi": 57,
"Secde Sûresi": 75,
"Ahzâb Sûresi": 90,
"Sebe' Sûresi": 58,
"Fâtır Sûresi": 43,
"Yâsin Sûresi": 41,
"Sâffât Sûresi": 56,
"Sâd Sûresi": 38,
"Zümer Sûresi": 59,
"Mü'min Sûresi": 60,
"Fussilet Sûresi": 61,
"Şûrâ Sûresi": 62,
"Zuhruf Sûresi": 63,
"Duhân Sûresi": 64,
"Câsiye Sûresi": 65,
"Ahkaf Sûresi": 66,
"Muhammed Sûresi": 95,
"Fetih Sûresi": 111,
"Hucurât Sûresi": 106,
"Kaf Sûresi": 34,
"Zâriyât Sûresi": 67,
"Tûr Sûresi": 76,
"Necm Sûresi": 23,
"Kamer Sûresi": 37,
"Rahmân Sûresi": 97,
"Vâkıa Sûresi": 46,
"Hadid Sûresi": 94,
"Mücâdele Sûresi": 105,
"Haşr Sûresi": 101,
"Mümtehine Sûresi": 91,
"Saf Sûresi": 109,
"Cum'a Sûresi": 110,
"Münâfikûn Sûresi": 104,
"Teğabün Sûresi": 108,
"Talâk Sûresi": 99,
"Tahrim Sûresi": 107,
"Mülk Sûresi": 77,
"Kalem Sûresi": 2,
"Hâkka Sûresi": 78,
"Meâric Sûresi": 79,
"Nuh Sûresi": 71,
"Cin Sûresi": 40,
"Müzzemmil Sûresi": 3,
"Müddessir Sûresi": 4,
"Kıyamet Sûresi": 31,
"İnsan Sûresi": 98,
"Mürselât Sûresi": 33,
"Nebe' Sûresi": 80,
"Nâziât Sûresi": 81,
"Abese Sûresi": 24,
"Tekvir Sûresi": 7,
"İnfitâr Sûresi": 82,
"Mutaffifin Sûresi": 86,
"İnşikak Sûresi": 83,
"Bürûc Sûresi": 27,
"Târık Sûresi": 36,
"A'lâ Sûresi": 8,
"Gâşiye Sûresi": 68,
"Fecr Sûresi": 10,
"Beled Sûresi": 35,
"Şems Sûresi": 26,
"Leyl Sûresi": 9,
"Duhâ Sûresi": 11,
"İnşirâh Sûresi": 12,
"Tin Sûresi": 28,
"Alak Sûresi": 1,
"Kadir Sûresi": 25,
"Beyyine Sûresi": 100,
"Zilzâl Sûresi": 93,
"Âdiyât Sûresi": 14,
"Kâria Sûresi": 30,
"Tekâsür Sûresi": 16,
"Asr Sûresi": 13,
"Hümeze Sûresi": 32,
"Fil Sûresi": 19,
"Kureyş Sûresi": 29,
"Mâûn Sûresi": 17,
"Kevser Sûresi": 15, 
"Kâfirûn Sûresi": 18,
"Nasr Sûresi": 114,
"Tebbet Sûresi": 6,
"İhlâs Sûresi": 22,
"Felâk Sûresi": 20,
"Nâs Sûresi": 21
}

In [ ]:
def surah_ordered(surahname):
    surahidx = surah_order.get(surahname)
    return surahidx

In [ ]:
df["Order"] = df["Name"].apply(surah_ordered)

In [ ]:
df.isnull().sum()

In [ ]:
def count_ayat(surah):
    ayat = df[df["Surah"] == surah][["Ayat"]].shape[0]
    return ayat

In [ ]:
df["Ayat Count"] = df["Surah"].apply(count_ayat)

In [ ]:
surah_ayat_count = df.groupby(by="Surah")[["Ayat Count", "Order"]].mean()

In [ ]:
surah_ayat_count.head()

In [ ]:
plt.figure(figsize=(20, 5))
ax = sns.barplot(data=surah_ayat_count, x=surah_ayat_count.index, y=surah_ayat_count["Ayat Count"])
ax.bar_label(ax.containers[0])
ax.set_title("Ayat Count per Surah")
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(20, 5))
ax = sns.barplot(data=surah_ayat_count, x=surah_ayat_count.index, y=surah_ayat_count["Ayat Count"], order=surah_ayat_count["Order"])
ax.bar_label(ax.containers[0])
ax.set_title("Ayat Count per Surah (ordered by Surah)")
plt.xticks(rotation=90)
plt.show()

# Preprocess

In [ ]:
def preprocess_text(text):
    text = text.lower()
    string.punctuation += "´"
    string.punctuation += "‘"
    string.punctuation += "’"
    text = text.translate(str.maketrans("", "", string.punctuation))
    words = text.split()
    words = [re.sub(r'(.)\1{1,}', r'\1\1', word) for word in words]
    words = [word.strip() for word in words if len(word.strip()) > 1 and word not in set(stopwords.words("english"))]
    text = " ".join(words)
    return text

In [ ]:
df["Translation1_clean"] = df["Translation1"].apply(preprocess_text)
df["Translation2_clean"] = df["Translation2"].apply(preprocess_text)
df["Translation3_clean"] = df["Translation3"].apply(preprocess_text)
df["Tafaseer1_clean"] = df["Tafaseer1"].apply(preprocess_text)
df["Tafaseer2_clean"] = df["Tafaseer2"].apply(preprocess_text)

In [ ]:
df.head()

In [ ]:
def word_freq(TEXTS, surahidx):
    freq_df = []
    for i in range(len(df[df["Surah"] == surahidx])):
        text = "".join(df[(df["Surah"] == surahidx & (df["Ayat"] == i+1))][TEXTS].values)
        freq_df.append(text)
        
    freq_words = [i.lower() for i in freq_df]
    freq_punc = []
    
    for o  in freq_words:
        freq_punc += nltk.word_tokenize(o)
        
    freq_func = [o for o in freq_punc if o not in string.punctuation]
    freq_freq = Counter(freq_punc)
    
    freq_top = freq_freq.most_common(15)
    
    words = [word for word, _ in freq_top]
    counts = [counts for _, counts in freq_top]
    
    plt.figure(figsize=(15, 10))
    ax = sns.barplot(y=words, x=counts)
    ax.bar_label(ax.containers[0])
    plt.xlabel("Frequency")
    plt.ylabel("Words")
    ax.set_title(f"Most common 15 words in {df[df['Surah'] == surahidx][['Name']].values[0][0]}")
    plt.show()
    
    return freq_top

In [ ]:
fatiha = word_freq("Tafaseer1_clean", 1)

In [ ]:
def print_wordcloud(dict_top):
    dict_top = dict(dict_top)
    wordcloud = WordCloud(width=250, height=250, background_color="black", min_font_size=5).generate_from_frequencies(dict_top)
    plt.figure()
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad=0)
    plt.show()

In [ ]:
print_wordcloud(fatiha)

# Similarity

In [ ]:
turkish = df["Tafaseer1_clean"].tolist()
tfidf = TfidfVectorizer()
uni_matrix = tfidf.fit_transform(turkish)
cos_sim = cosine_similarity(uni_matrix)

In [ ]:
def recommend_texts(text, n=5):
    text_index = df[df["Tafaseer1_clean"] == text].index[0]
    text_similarities = cos_sim[text_index]
    top_indices = text_similarities.argsort()[::-1][1:n+1]
    recommended_text = df.loc[top_indices]
    return recommended_text

In [ ]:
recommend_texts("name god compassionate merciful")

# Summary

In [ ]:
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

In [ ]:
test = "".join(df[(df["Surah"] == 1 & (df["Ayat"] == 2))]["Tafaseer1"].values)
test

In [ ]:
inputs = bart_tokenizer([test], max_length=1024, return_tensors="pt")
summary_ids = bart_model.generate(inputs['input_ids'], num_beams=4, max_length=100, early_stopping=True)
summary = ([bart_tokenizer.decode(i, skip_special_tokens=True, clean_up_tokenization_spaces=False) for i in summary_ids])
print(summary)

# Sentiment Analysis

In [ ]:
sentiment = SentimentIntensityAnalyzer()

In [ ]:
def sentiment_score(TEXTS, surahidx):
    total_neg, total_neu, total_pos = 0, 0, 0
    ayats_len = len(df[df["Surah"] == surahidx])
    for i in range(ayats_len):
        text = "".join(df[(df["Surah"] == surahidx & (df["Ayat"] == i+1))][TEXTS].values)
        score = sentiment.polarity_scores(text)
        total_neg += score.get("neg") / ayats_len
        total_neu += score.get("neu") / ayats_len
        total_pos += score.get("pos") / ayats_len
        
        print(i+1, sentiment.polarity_scores(text))
        
    scores = [total_neg, total_neu, total_pos]
    labels = ["NEGATIVE", "NEUTRAL", "POSITIVE"]
    
    maxi_idx = scores.index(max(scores))
    res = labels[maxi_idx]
    print(f"The surah {df[df['Surah'] == surahidx][['Name']].values[0][0]} is {res}, score: {max(scores)}")

In [ ]:
sentiment_score("Translation1", 1)